# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`


In [ ]:
# Import dependencies
from pymongo import MongoClient
import json
import requests 
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [ ]:
# review the collections in our new database
print(uk_food.list_collection_names())

In [ ]:
# review a document in the establishments collection
pprint(uk_food.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = uk_food['establishments']

# Print the number of documents in collection
queryall = {}
print("Number of all documents in collection:", establishments.count_documents(queryall))

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data

new_rest = {"BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True}


# Insert the new restaurant into the collection
establishments.insert_one(new_rest)

queryall = {}
# Print the number of results
print("Number of all documents in collection:", establishments.count_documents(queryall))

In [ ]:
# Check that the new restaurant was inserted
uk_food.establishments.find({'BusinessName':'Penang Flavour'})

query = {'BusinessName': "Penang Flavours"}
results = establishments.find(query)
for result in results:
    print(result)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessType': "Restaurant/Cafe/Canteen"}
fields = {'BusinessType': 1, 'BusinessTypeID': 1 }
# Capture the results to a variable
results = establishments.find(query, fields)

# Pretty print the results
#Top 3 printed to confirm consistant result 

for i in range(3):
 pprint(results[i])


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID

uk_food.establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
        {'BusinessTypeID': 1}
     }
)


In [ ]:
# Confirm that the new restaurant was updated
# check the 'BusinessTypeID'is changed to 1
query = {'BusinessName': "Penang Flavours"}
results = establishments.find(query)
for result in results:
    print(result)


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"

# Create a query that finds the documents that have a "parameter" of "Dover" in the establishments collection
query = {'LocalAuthorityName': 'Dover'}

# Print the number of results
print("Number of documents in result:", establishments.count_documents(query))



In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"

query_dover = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query_dover)

# Print the number of results
queryall = {}
print("Number of all documents in collection after Dover deletion:", establishments.count_documents(queryall))

In [ ]:
# Check if any remaining documents include Dover

# Create a query that finds the documents that have a "parameter" of "Dover" in the establishments collection
query_dover = {'LocalAuthorityName': 'Dover'}

# Print the number of results
print("Number of documents containing a local authority of Dover in result:", establishments.count_documents(query_dover))


In [ ]:
# Check that other documents remain with 'find_one'
pprint(uk_food.establishments.find_one())

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude

establishments.update_many({}, [ {'$set': { "geocode.longitude" : {'$toDouble': "$geocode.longitude"}
                                              } 
                                     } ]
                              )


In [ ]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [ {'$set': { "geocode.latitude" : {'$toDouble': "$geocode.latitude"}
                                              } 
                                     } ]
                              )

In [ ]:
# Check that the coordinates are now numbers
pprint(uk_food.establishments.find_one())